<a href="https://colab.research.google.com/github/Francrodi/AprendAut_T2/blob/master/Cliente_Lab2_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 2 - Aprendizaje Bayesiano

### Grupo 50:
    - M. Ramilo (5.665.788-5)
    - G. Dinello (5.031.022-5)
    - F. Crocamo (4.732.816-6)

## 1. Objetivo

El objetivo central de esta tarea es crear un **predictor de palabras** que funcione de manera similar a las sugerencias de palabras que encontramos en los teclados de dispositivos celulares, y para lograrlo, vamos a emplear el algoritmo **Naive Bayes**. Para entrenar este modelo, utilizaremos un chat extenso extraído de **WhatsApp**. Esta elección de datos presenta desafíos particulares de preprocesamiento debido a la naturaleza informal y variada de las conversaciones en WhatsApp.

En esencia, estamos buscando desarrollar una herramienta que sea capaz de **predecir la siguiente palabra** basándose en el contexto de lo que se ha escrito previamente. Esto puede ser especialmente útil en la escritura de mensajes rápidos y eficientes en dispositivos móviles, ya que ahorra tiempo al usuario al sugerir las palabras más probables de acuerdo con el contexto.

## 2. Preprocesamiento

Como se mencionó anteriormente, esta sección del proyecto se centra en la etapa de preprocesamiento de los datos, que es una fase fundamental para optimizar el chat original extraído de WhatsApp y prepararlo adecuadamente para su uso en el entrenamiento del modelo de predicción de palabras. Esta etapa desempeña un papel esencial en la calidad y eficacia del predictor de palabras que estamos desarrollando.

### 2.1 Cargamos el chat

In [ ]:
filename = "chat.txt"

with open(filename, "r", encoding="utf-8") as file:
  data = file.readlines()

raw_chat = []

for line in data:
  parts = line.split(":")
  message = parts[-1].strip()
  raw_chat.append(message)

### 2.2 Preprocesamos el chat

Preprocesamos el chat para eliminar todos los elementos que disminuyen la calidad del predictor que queremos construir.

Los elementos filtrados son:

*   URLs
*   Menciones ("@alguien")
*   Risas
*   Caracteres especiales
*   Palabras con letras repetidas ("vamosss", "biennnn", etc...)

Se aplicaron transformaciones a abreviaciones de palabras para mantener consistencia en predicciones. Algunos ejemplos son:

*   "tmb" a "también"
*   "xq" a "porque"
*   "q" a "que"


In [ ]:
import re

# Expresiones regulares
URL = r"https?://\S+"
MENTION = r"@\S+"
LAUGH = r"(?:[aA]*(?:[jJ][aA]*)+[jJ]?)\b"
NOT_LETTERS = r"[^a-zñáéíóú\s]"
REPEATED_LETTERS = r"([a-zñáéíóú])\1{2,}"

# Mapeo de palabras informales a palabras formales
INFORMAL_TO_FORMAL = {
  "q": "que",
  "ke": "que",
  "k": "que",
  "xq": "porque",
  "porq": "porque",
  "pq": "porque",
  "p": "para",
  "pa": "para",
  "d": "de",
  "n": "en",
  "tb": "también",
  "tmb": "también",
  "tmbn": "también",
  "tmbien": "también",
  "tmbn": "también",
  "tambn": "también",
  "toy": "estoy",
  "tas": "estas",
  "ta": "está",
  "tan": "estan",
  "tamos": "estamos",
  "tamo": "estamos",
  "pal": "para el",
  "pa la": "para la",
  "pa las": "para las",
  "pa los": "para los",
  "vamo": "vamos",
  "bue": "bueno",
  "boe": "bueno",
  "buee": "bueno",
  "bn": "bien",
}


def preprocess_chat(chat: list) -> list:
  """Preprocesa el chat de WhatsApp, convirtiendo todas las letras en minusculas, eliminando URLs, menciones, risas, caracteres especiales, letras repetidas y mensajes vacios. Reemplaza palabras informales por sus equivalentes formales. Finalmente, ordena los mensajes alfabeticamente y elimina duplicados.

  Args:
    chat (list): Lista de mensajes del chat de WhatsApp

  Returns:
    list: Lista de mensajes preprocesados
  """
  chat = [message.lower() for message in chat]

  chat = [re.sub(URL, "", message) for message in chat]
  chat = [re.sub(MENTION, "", message) for message in chat]
  chat = [re.sub(NOT_LETTERS, "", message) for message in chat]
  chat = [re.sub(LAUGH, "", message) for message in chat]
  chat = [re.sub(REPEATED_LETTERS, r"\1", message) for message in chat]

  for informal, formal in INFORMAL_TO_FORMAL.items():
    chat = [re.sub(r"\b{}\b".format(informal), formal, message) for message in chat]

  chat = [message.strip() for message in chat]
  chat = [message for message in chat if message != ""]

  chat = [re.sub(r"\s+", " ", message) for message in chat]
  chat = list(dict.fromkeys(chat))

  return chat

chat = preprocess_chat(raw_chat)
chat

['los mensajes y las llamadas están cifrados de extremo a extremo nadie fuera de este chat ni siquiera whatsapp puede leerlos ni escucharlos toca para obtener más información',
 'creó el grupo métodos numéricos fing',
 'cualquier miembro de esta comunidad puede unirse a este grupo',
 'te uniste a un grupo en la comunidad matemática fing mediante una invitación',
 'se unió desde la comunidad',
 'se unió usando el enlace de invitación de este grupo',
 'buenas a alguien le falta grupo con un amigo somos',
 'a mi',
 'a mi también',
 'grupo para que estoy re perdido',
 'el lab es en grupo',
 'se',
 'mañana ya hay clase',
 'es de a cuatro el lab no',
 'o',
 'exacto',
 'yo estoy en un grupo de',
 'si queres los juntamos',
 'estoy sin grupo por si alguien quiere armar',
 'yo estoyy',
 'ya hay que armar los grupos',
 'cuando quieras pero ya se sabe que las actividades son en grupo',
 'fa bueno yo tampoco tengo grupo asi que si alguien quiere armamos',
 'estoy',
 'el trabajo obligatorio es en gr

## 3. Estructuras de datos

Para lograr un funcionamiento eficiente debemos crear las estructuras de datos adecuadas para el problema a resolver. A la hora de correr el predictor, necesitaremos tiempos de busqueda casi instantaneos, de otra manera el predictor en lugar de ayudar al usuario lo estaria trancando y estorbando.

Para lograr esto, utilizaremos **tablas de hash (diccionarios en Python)** para almacenar las probabilidades y cantidad de ocurrencia de palabras, esto nos permite realizar las busquedas en **O(1)**.

Por otro lado, construiremos una lista para almacenar el vocabulario, ya que en cada prediccion a realizar tendremos que recorrer todo el vocabulario en busca de la palabra mas probable (Hipotesis MAP).

De esta forma podemos conseguir tiempos de prediccion de **O(N)** siendo N la cantidad de palabras en el vocabulario.

### 3.1 Cargamos diccionario de palabras en español

El mismo sera utilizado para cruzarlo con nuestro vocabulario generado a partir del chat y asi eliminar palabras no deseadas.

In [ ]:
filename = "es.txt"

with open(filename, "r", encoding="utf-8") as file:
   data = file.readlines()

spanish_dict = set()

for line in data:
   word = line.strip()
   spanish_dict.add(word)

spanish_dict.add('y')

### 3.2 Creamos diccionario de ocurrencia de palabras y vocabulario

In [ ]:
def build_word_count(chat, spanish_dict=None):
    words_list = " ".join(chat).split(" ")

    if spanish_dict is not None:
        # filtrado de palabras con diccionario
        filtered_words_list = [word for word in words_list if word in spanish_dict]
    else:
        filtered_words_list = words_list

    # conteo de ocurrencias de cada palabra
    words_count = {}
    # for word in filtered_words_list:
    for word in filtered_words_list:
        if word in words_count:
            words_count[word] += 1
        else:
            words_count[word] = 1

    return words_count

def build_vocabulary(words_count):
    return list(words_count.keys())

In [ ]:
words_count

{'los': 150,
 'mensajes': 4,
 'y': 389,
 'las': 178,
 'llamadas': 1,
 'están': 12,
 'cifrados': 1,
 'de': 679,
 'extremo': 2,
 'a': 422,
 'nadie': 9,
 'fuera': 4,
 'este': 36,
 'chat': 3,
 'ni': 33,
 'siquiera': 6,
 'whatsapp': 1,
 'puede': 47,
 'leerlos': 1,
 'escucharlos': 1,
 'toca': 2,
 'para': 196,
 'obtener': 6,
 'más': 65,
 'información': 3,
 'creó': 1,
 'el': 673,
 'grupo': 102,
 'métodos': 13,
 'numéricos': 5,
 'fing': 7,
 'cualquier': 7,
 'miembro': 1,
 'esta': 76,
 'comunidad': 6,
 'unirse': 1,
 'te': 146,
 'uniste': 1,
 'un': 139,
 'en': 434,
 'la': 508,
 'matemática': 2,
 'mediante': 1,
 'una': 130,
 'invitación': 3,
 'se': 187,
 'unió': 3,
 'desde': 9,
 'usando': 12,
 'enlace': 2,
 'buenas': 72,
 'alguien': 85,
 'le': 78,
 'falta': 25,
 'con': 168,
 'amigo': 13,
 'somos': 18,
 'mi': 59,
 'también': 43,
 'que': 822,
 'estoy': 55,
 're': 22,
 'perdido': 6,
 'lab': 2,
 'es': 343,
 'mañana': 21,
 'ya': 69,
 'hay': 75,
 'clase': 40,
 'cuatro': 1,
 'no': 458,
 'o': 134,
 'exact

### 3.3 Definimos funciones para manejar los diccionarios de probabilidades

Definimos las funciones necesarias para construir y actualizar los diccionarios que almacenan las probabilidades de **P(H)** y **P(D|H)**.

In [ ]:
# Construimos diccionario de probabilidades P(H), probabilidad a priori
def build_ph(total_words) -> dict[str, float]:
    ph_dict = {word: frequency / total_words for word, frequency in words_count.items()}
    return ph_dict

def build_pdh(chat: list, N=1) -> dict[str, dict[str, float]]:
    pdh_dict = {}
    pdh_dict_count = {}
    for message in chat:
        words = message.split(" ")
        words = [word for word in words if word in vocabulary]
        if len(words) > 1:
            for key, val in enumerate(words):
                if (val not in pdh_dict_count.keys()):
                    pdh_dict[val] = {}
                    pdh_dict_count[val] = {}
                if key >= 1:
                    if key-N < 0:
                        initial = 0
                    else:
                        initial = key-N
                    N_words_before = set(words[initial:key])
                    for word_before in N_words_before:
                        if not word_before in pdh_dict_count[val]:
                            pdh_dict[val][word_before] = 0
                            pdh_dict_count[val][word_before] = 0
                        pdh_dict_count[val][word_before] += 1

    for key in pdh_dict.keys():
        for subkey in pdh_dict[key].keys():
            pdh_dict[key][subkey] = pdh_dict_count[key][subkey] / words_count[subkey]

    return pdh_dict, pdh_dict_count


def update_pdh(pdh_dict, pdh_dict_count, new_message, N=1):
    words = new_message.split(" ")
    words = [word for word in words if word in vocabulary]
    if len(words) > 1:
        for key, val in enumerate(words):
            if (val not in pdh_dict_count.keys()):
                pdh_dict_count[val] = {}
                pdh_dict[val] = {}
            if key >= 1:
                if key-N < 0:
                    initial = 0
                else:
                    initial = key-N
                N_words_before = set(words[initial:key])
                for word_before in N_words_before:
                    if not word_before in pdh_dict_count[val]:
                        pdh_dict_count[val][word_before] = 0
                        pdh_dict[val][word_before] = 0
                    pdh_dict_count[val][word_before] += 1

    for key in pdh_dict.keys():
        for subkey in pdh_dict[key].keys():
            pdh_dict[key][subkey] = pdh_dict_count[key][subkey] / words_count[subkey]

    return pdh_dict, pdh_dict_count


def train(pdh_dict, pdh_dict_count, N=2, chat=chat, spanish_dict=spanish_dict):
    global total_words, vocabulary, words_count
    words_count = build_word_count(chat, spanish_dict=spanish_dict)
    print("1")
    total_words = sum(words_count.values())
    print("2")
    vocabulary = build_vocabulary(words_count)
    print("3")
    ph = build_ph(total_words)
    print("4")
    if pdh_dict_count is None:
        pdh_dict, pdh_dict_count = build_pdh(chat, N=N)
    else:
        pdh_dict, pdh_dict_count = update_pdh(pdh_dict, pdh_dict_count, chat[-1], N=N)
    print("5")
    return ph, pdh_dict, pdh_dict_count

## 4. Implementación

### 4.1 Creamos las estructuras necesarias

Definimos las stopwords, el hiperparametro N y creamos los diccionarios de probabilidades **P(H)** y **P(D|H)**.

In [ ]:
import nltk
nltk.download('stopwords')

# Definimos stopwords
stopwords = nltk.corpus.stopwords.words("spanish")
stopwords.append("no")
stopwords.append("si")
stopwords.append("mas")
stopwords.append("que")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'los': {'hay': 0.04,
  'que': 0.021897810218978103,
  'armar': 0.25,
  'de': 0.048600883652430045,
  'ejercicios': 0.16666666666666666,
  'y': 0.02313624678663239,
  'gracias': 0.018518518518518517,
  'a': 0.02132701421800948,
  'todos': 0.32142857142857145,
  'van': 0.0625,
  'pedir': 0.2857142857142857,
  'como': 0.015748031496062992,
  'pasó': 0.08333333333333333,
  'con': 0.05357142857142857,
  'docente': 0.2,
  'pero': 0.011627906976744186,
  'entregar': 0.2,
  'necesitan': 1.0,
  'una': 0.015384615384615385,
  'mano': 0.09090909090909091,
  'las': 0.011235955056179775,
  'clases': 0.1,
  'del': 0.0196078431372549,
  'otro': 0.08695652173913043,
  'lado': 0.125,
  'son': 0.078125,
  're': 0.045454545454545456,
  'buenos': 1.0,
  'tarde': 0.125,
  'unos': 0.14285714285714285,
  'crack': 0.5,
  'script': 0.0625,
  'pido': 1.0,
  'habiendo': 0.5,
  'antes': 0.05263157894736842,
  'definido': 0.3333333333333333,
  'trabajo': 0.25,
  'o': 0.014925373134328358,
  'sea': 0.1,
  'hacen':

In [ ]:
N=3
ph_dict, pdh_dict, pdh_dict_count = train(None, None, N=N, chat=chat, spanish_dict=spanish_dict)

### 4.2 Creacion del predictor

Construimos la funcion `recomendacion_bayesiana` que dada una frase devuelve la palabra sugerida. Para esto calculamos la **Hipotesis MAP** utilizando el **m-estimador** para los casos que nunca se habian visto en el entrenamiento del modelo.

In [ ]:

def recomendacion_bayesiana(frase):
  if len(frase) >= N:
    keys = frase[-N:]
  else:
    keys = frase

  keys = [k.lower() for k in keys]

  # Bucar la palabra con mayor probabilidad, es decir, que maximice P(H|D) = P(D|H) * P(H) (Hipoteses MAP)
  max_prob = 0
  palabra_sugerida = ""
  last_is_stopword = True if keys[-1] in stopwords else False
  for word in vocabulary:
    if last_is_stopword and word in stopwords:
      continue
    prob = None
    for key in keys:
      m_estimator = 1 / (words_count.get(key, 0) + len(vocabulary))
      prob = pdh_dict.get(word, {}).get(key, m_estimator) if prob is None else prob * pdh_dict.get(word, {}).get(key, m_estimator)
    prob *= ph_dict[word]
    if prob >= max_prob:
      max_prob = prob
      palabra_sugerida = word

  return palabra_sugerida


##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")
frase = []
palabra_sugerida = ""
while 1:
    palabra = input(">> ")

    if palabra == "..":
      break

    elif palabra == ".":
      print("----- Comenzando frase nueva -----")
      chat.append(" ".join(frase))
      ph_dict, pdh_dict, pdh_dict_count = train(pdh_dict, pdh_dict_count, N=N, chat=chat, spanish_dict=spanish_dict)
      frase = []

    elif palabra == "": # acepta última palabra sugerida
      frase.append(palabra_sugerida)

    else: # escribió una palabra
      frase.append(palabra)

    if frase:
      palabra_sugerida = recomendacion_bayesiana(frase)

      frase_propuesta = frase.copy()
      frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

      print(" ".join(frase_propuesta))


Ingrese la frase dando ENTER luego de cada palabra.
Ingrese sólo ENTER para aceptar la recomendación sugerida, o escriba la siguiente palabra y de ENTER
Ingrese '.' para comenzar con una frase nueva.
Ingrese '..' para terminar el proceso.


KeyboardInterrupt: ignored

# ideas

- evaluacion tomando 1000 frases random de N+1 palabras del dataset y evaluando la prediccion basado en las N

In [ ]:
import numpy as np
import random
np.random.seed(42)
random.seed(42)

# hiperparametro N
N = 4
ph = build_ph()
pdh = build_pdh(chat, N)

shuffled_chats = chat.copy()
random.shuffle(shuffled_chats)
## Preparamos el test set
test_set = []
# Elegimos 1000 mensajes al azar
index = 0
while len(test_set) < 1000 and index < len(shuffled_chats):
    msg = shuffled_chats[index]
    words = msg.split(" ")
    index += 1
    if len(words) <= N:
        continue
    # elijo la word que tengo que predecir dentro del mensaje
    to_predict = np.random.randint(N, len(words))
    context = words[to_predict-N:to_predict]
    # N=3
    # words = ["a","b","c","d"]
    # to_predict = 3<= rand < 4
    # context = words[3-3:3] = words[0:3] = ["a","b","c"]
    test_set.append((context, words[to_predict]))

len(test_set)

In [ ]:
correctas = 0
for context, word in test_set:
    prediction = recomendacion_bayesiana(context)
    correctas += 1 if prediction == word else 0

print(correctas / len(test_set))